In [22]:
from __future__ import absolute_import, division, print_function, unicode_literals
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import absl.logging

absl.logging.set_verbosity(absl.logging.ERROR)

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

# On Mac you may encounter an error related to OMP, this is a workaround, but slows down the code
# https://github.com/dmlc/xgboost/issues/1715
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

In [ ]:
import tensorflow as tf

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.__version__

In [ ]:
if tf.test.gpu_device_name():
    print("Default GPU Device:{}".format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF if you have one.")

In [ ]:
!pip install gitpython

import git
repo_url = 'https://github.com/isl-org/OpenBot.git'  # Replace with your GitHub repository URL
repo_destination = '/content/openbot'  # Destination folder in Colab

# Function to check if a folder is empty
def is_folder_empty(folder_path):
    return not os.path.exists(folder_path) or not os.listdir(folder_path)


if is_folder_empty(repo_destination):
  git.Repo.clone_from(repo_url, repo_destination)
  cloned_contents = os.listdir(repo_destination)
  print("Cloned Repository Contents:", cloned_contents)
else:
    print("Destination folder is not empty. Skipping cloning.")

In [26]:
from openbot.policy.openbot import dataloader, data_augmentation, utils, train

## Set train and test dirs

Define the dataset directory and give it a name. Inside the dataset folder, there should be two folders, `train_data` and `test_data`.

In [27]:
dataset_dir = "dataset"
dataset_name = "openbot"
train_data_dir = os.path.join(dataset_dir, "train_data")
test_data_dir = os.path.join(dataset_dir, "test_data")

## Add train and test data

In [ ]:
from google.colab import files
import zipfile

UPLOAD_FOLDER_TRAIN = '/content/openbot/policy/dataset/train_data/my_dataset_1'
UPLOAD_FOLDER_TEST = '/content/openbot/policy/dataset/test_data/my_dataset_2'
ALLOWED_EXTENSIONS = {'zip'}
UPLOAD_FOLDER_TRAIN_TEXT = "Upload train zip file"
UPLOAD_FOLDER_TEST_TEXT = "Upload test zip file"

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def upload_and_extract(upload_folder,upload_folder_text):

    print(upload_folder_text)
    uploaded = files.upload()

    if len(uploaded) == 0:
        print("No file uploaded.")
        return

    filename = list(uploaded.keys())[0]

    if allowed_file(filename):
        zip_folder = os.path.splitext(filename)[0]
        zip_folder_path = os.path.join(upload_folder, zip_folder)
        os.makedirs(zip_folder_path, exist_ok=True)

        zip_filepath = os.path.join(zip_folder_path, filename)
        with open(zip_filepath, 'wb') as f:
            f.write(uploaded[filename])

        with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
            zip_ref.extractall(zip_folder_path)

        os.remove(zip_filepath)

        print(f"File '{filename}' successfully uploaded and extracted to '{zip_folder_path}'.")
    else:
        print("Invalid file format. Please upload a valid ZIP file.")

upload_and_extract(UPLOAD_FOLDER_TRAIN,UPLOAD_FOLDER_TRAIN_TEXT)
upload_and_extract(UPLOAD_FOLDER_TEST,UPLOAD_FOLDER_TEST_TEXT)


## Hyperparameters



You may have to tune the learning rate and batch size depending on your available compute resources and dataset. As a general rule of thumb, if you increase the batch size by a factor of n, you can increase the learning rate by a factor of sqrt(n). In order to accelerate training and make it more smooth, you should increase the batch size as much as possible. In our paper we used a batch size of 128. For debugging and hyperparamter tuning, you can set the number of epochs to a small value like 10. If you want to train a model which will achieve good performance, you should set it to 50 or more. In our paper we used 100.

In [29]:
params = train.Hyperparameters()

params.MODEL = "pilot_net"  # choices: "pilot_net","cil_mobile","cil_mobile_fast","cil"
params.POLICY = "autopilot"  # choices: "autopilot","point_goal_nav"
params.TRAIN_BATCH_SIZE = 128
params.TEST_BATCH_SIZE = 16
params.LEARNING_RATE = 0.0003
params.NUM_EPOCHS = 100
params.BATCH_NORM = True  # use batch norm (recommended)
params.FLIP_AUG = False  # flip image and controls as augmentation (only autopilot)
params.CMD_AUG = False  # randomize high-level command as augmentation (only autopilot)
params.USE_LAST = False  # resume training from last checkpoint
params.WANDB = False
# policy = "autopilot": images are expected to be 256x96 - no cropping required
# policy = "point_goal_nav": images are expected to be 160x120 - cropping to 160x90
params.IS_CROP = params.POLICY == "point_goal_nav"

## Pre-process the dataset

In [30]:
tr = train.Training(params)
tr.dataset_name = dataset_name
tr.train_data_dir = os.path.join("/content","openbot", "policy", "dataset", "train_data")
tr.test_data_dir = os.path.join("/content","openbot", "policy", "dataset", "test_data")

Running this for the first time will take some time. This code will match image frames to the controls (labels) and indicator signals (commands).  By default, data samples where the vehicle was stationary will be removed. If this is not desired, you need to set `tr.remove_zeros = False`. If you have made any changes to the sensor files, changed `remove_zeros` or moved your dataset to a new directory, you need to set `tr.redo_matching = True`.

In [ ]:
tr.redo_matching = False
tr.remove_zeros = True
train.process_data(tr)

In [32]:
import threading


def broadcast(event, payload=None):
    print(event, payload)


event = threading.Event()
my_callback = train.MyCallback(broadcast, event)

In the next step, you can convert your dataset to a tfrecord, a data format optimized for training. You can skip this step if you already created a tfrecord before or if you want to train using the files directly.

In [ ]:
train.create_tfrecord(my_callback, policy=tr.hyperparameters.POLICY)

## Load the dataset

If you did not create a tfrecord and want to load and buffer files from disk directly, set `no_tf_record = True`.

In [34]:
no_tf_record = False

In [ ]:
if no_tf_record:
    tr.train_data_dir = train_data_dir
    tr.test_data_dir = test_data_dir
    train.load_data(tr, verbose=0)
else:
    tr.train_data_dir = os.path.join(dataset_dir, "/content/openbot/policy/dataset/tfrecords/train.tfrec")
    tr.test_data_dir = os.path.join(dataset_dir, "/content/openbot/policy/dataset/tfrecords/test.tfrec")
    train.load_tfrecord(tr, verbose=0)

models_dir = "/content/openbot/policy/models"
model_dir = os.path.join(models_dir, tr.model_name)
os.makedirs(model_dir, exist_ok=True)  # Create model directory if it doesn't exist

In [ ]:
# Select interactive backend to show inline
%matplotlib inline
utils.show_batch(dataset=tr.train_ds, policy=tr.hyperparameters.POLICY, model=None)

## Training

The number of epochs is proportional to the training time. One epoch means going through the complete dataset once. Increasing `NUM_EPOCHS` will mean longer training time, but generally leads to better performance. To get familiar with the code it can be set to small values like `5` or `10`. To train a model that performs well, it should be set to values between `50` and `200`. Setting `USE_LAST` to `true` will resume the training from the last checkpoint. The default values are `NUM_EPOCHS = 100` and `USE_LAST = False`. They are set in [Hyperparameters](#hyperparameters).

In [ ]:
# params.NUM_EPOCHS = 10
# params.USE_LAST = True

train.do_training(tr, my_callback, verbose=1)

## Evaluation

The loss and mean absolute error should decrease. This indicates that the model is fitting the data well. The custom metrics (direction and angle) should go towards 1. These provide some additional insight to the training progress. The direction metric measures weather or not predictions are in the same direction as the labels. Similarly the angle metric measures if the prediction is within a small angle of the labels. The intuition is that driving in the right direction with the correct steering angle is most critical part for good final performance.

### Plot metrics

In [ ]:
x = np.arange(tr.INITIAL_EPOCH + 1, tr.history.params["epochs"] + 1, 1)

In [ ]:
plt.figure().gca().xaxis.get_major_locator().set_params(integer=True)
plt.plot(x, tr.history.history["loss"], label="loss")
plt.plot(x, tr.history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.savefig(os.path.join(tr.log_path, "loss.png"), bbox_inches="tight")

In [ ]:
plt.figure().gca().xaxis.get_major_locator().set_params(integer=True)
plt.plot(x, tr.history.history["mean_absolute_error"], label="mean_absolute_error")
plt.plot(
    x, tr.history.history["val_mean_absolute_error"], label="val_mean_absolute_error"
)
plt.xlabel("Epoch")
plt.ylabel("Mean Absolute Error")
plt.legend(loc="upper right")
plt.savefig(os.path.join(tr.log_path, "error.png"), bbox_inches="tight")

In [ ]:
plt.figure().gca().xaxis.get_major_locator().set_params(integer=True)
plt.plot(x, tr.history.history["direction_metric"], label="direction_metric")
plt.plot(x, tr.history.history["val_direction_metric"], label="val_direction_metric")
plt.xlabel("Epoch")
plt.ylabel("Direction Metric")
plt.legend(loc="lower right")
plt.savefig(os.path.join(tr.log_path, "direction.png"), bbox_inches="tight")

In [ ]:
plt.figure().gca().xaxis.get_major_locator().set_params(integer=True)
plt.plot(x, tr.history.history["angle_metric"], label="angle_metric")
plt.plot(x, tr.history.history["val_angle_metric"], label="val_angle_metric")
plt.xlabel("Epoch")
plt.ylabel("Angle Metric")
plt.legend(loc="lower right")
plt.savefig(os.path.join(tr.log_path, "angle.png"), bbox_inches="tight")

### Save tf lite models for best train, best val and last checkpoint

In [ ]:
best_train_checkpoint = "cp-best-train.ckpt"
best_train_tflite = utils.generate_tflite(tr.checkpoint_path, best_train_checkpoint)
utils.save_tflite(best_train_tflite, tr.checkpoint_path, "best-train")
best_train_index = np.argmin(np.array(tr.history.history["loss"]))
print(
    "Best Train Checkpoint (epoch %s) - angle: %.4f, val_angle: %.4f, direction: %.4f, val_direction: %.4f"
    % (
        best_train_index,
        tr.history.history["angle_metric"][best_train_index],
        tr.history.history["val_angle_metric"][best_train_index],
        tr.history.history["direction_metric"][best_train_index],
        tr.history.history["val_direction_metric"][best_train_index],
    )
)

In [ ]:
best_val_checkpoint = "cp-best-val.ckpt"
best_val_tflite = utils.generate_tflite(tr.checkpoint_path, best_val_checkpoint)
utils.save_tflite(best_val_tflite, tr.checkpoint_path, "best")
utils.save_tflite(best_val_tflite, tr.checkpoint_path, "best-val")
best_val_index = np.argmin(np.array(tr.history.history["val_loss"]))
print(
    "Best Val Checkpoint (epoch %s) - angle: %.4f, val_angle: %.4f, direction: %.4f, val_direction: %.4f"
    % (
        best_val_index,
        tr.history.history["angle_metric"][best_val_index],
        tr.history.history["val_angle_metric"][best_val_index],
        tr.history.history["direction_metric"][best_val_index],
        tr.history.history["val_direction_metric"][best_val_index],
    )
)

In [ ]:
last_checkpoint = "cp-last.ckpt"
last_tflite = utils.generate_tflite(tr.checkpoint_path, last_checkpoint)
utils.save_tflite(last_tflite, tr.checkpoint_path, "last")
print(
    "Last Checkpoint - angle: %.4f, val_angle: %.4f, direction: %.4f, val_direction: %.4f"
    % (
        tr.history.history["angle_metric"][-1],
        tr.history.history["val_angle_metric"][-1],
        tr.history.history["direction_metric"][-1],
        tr.history.history["val_direction_metric"][-1],
    )
)

### Evaluate the best model (train loss) on the training set

In [ ]:
best_train_model = utils.load_model(
    os.path.join(tr.checkpoint_path, best_train_checkpoint),
    tr.loss_fn,
    tr.metric_list,
    tr.custom_objects,
)
loss, mae, direction, angle = best_train_model.evaluate(
    tr.train_ds,
    steps=tr.image_count_train / tr.hyperparameters.TRAIN_BATCH_SIZE,
    verbose=1,
)

In [ ]:
utils.show_batch(
    dataset=tr.train_ds, policy=tr.hyperparameters.POLICY, model=best_train_model
)

In [ ]:
utils.compare_tf_tflite(best_train_model, best_train_tflite)

### Evaluate the best model (val loss) on the validation set

In [ ]:
best_val_model = utils.load_model(
    os.path.join(tr.checkpoint_path, best_val_checkpoint),
    tr.loss_fn,
    tr.metric_list,
    tr.custom_objects,
)
loss, mae, direction, angle = best_val_model.evaluate(
    tr.test_ds,
    steps=tr.image_count_test / tr.hyperparameters.TEST_BATCH_SIZE,
    verbose=1,
)

In [ ]:
utils.show_batch(
    dataset=tr.test_ds, policy=tr.hyperparameters.POLICY, model=best_val_model
)

In [ ]:
utils.compare_tf_tflite(best_val_model, best_val_tflite)

## Save the notebook as HTML

In [ ]:
utils.save_notebook()
current_file = "policy_learning.ipynb"
output_file = os.path.join(tr.log_path, "notebook.html")
utils.output_HTML(current_file, output_file)